In [1]:
# Numpy is used to store the data in the form of arrays and apply some mathematical functionality to it
import numpy as np
# Counter in collections is used to store the number of iterable objects in the form of dictionary
from collections import Counter

In [2]:
# Creates a node for a tree
class Node:
    # By default all the attributes are passed as None
    def __init__(self, feature=None, threshold=None, left=None, right=None, current=None):
        # Index value of the feature which is been used for splitting
        self.feature = feature
        # Threshold using which we are going to split the data
        self.threshold = threshold
        # References to the left child node
        self.left = left
        # References to the right child node
        self.right = right
        # Current predicted class
        self.current = current

In [3]:
# Main class where we are going to prepare a decision tree
class DecisionTree:
    # the init function will have all the hyperparameters
    def __init__(self, maxDepth=100, minSplit=2):
        # Define the maximum depth of the tree just to avoid overfitting
        self.maxDepth = maxDepth
        # Minimum number of samples required so perform split operation
        self.minSplit = minSplit
        # The root node of the tree
        self.root = None

    # Method to initiate the building of the tree
    def fit(self, X, y):
        # The root gets the first node
        self.root = self.growTree(X, y)
        print("\nFinal Decision Tree Structure:")
        self.printTree(self.root)

    # Method to build the tree
    def growTree(self, X, y, depth=0):
        # Number if total samples and features present
        nSamples, nFeatures = X.shape
        # Number of unique output present
        nLabels = len(np.unique(y))
        print(f"\n{'  ' * depth}Depth {depth}:")
        print(f"{'  ' * depth}Labels: {y}")

        # Print entropy of the current node
        nodeEntropy = self.entropy(y)
        print(f"{'  ' * depth}Node Entropy: {nodeEntropy:.4f}")

        # Stopping criteria
        # 1. First condition checks whether the tree contructed has reached to its maximum depth
        # 2. Second condition checks whether the node is pure or not i.e., only one possible outcome
        # 3. Third condition checks if the number of sample in the given node are less than minimum of samples defined
        if (depth >= self.maxDepth or nLabels == 1 or nSamples < self.minSplit):
            # It will create a dictionary with unique outputs as keys and their total number of occurences as the values
            counter = Counter(y)
            # Extract the key with largest value assigned to it
            leafValue = counter.most_common(1)[0][0]
            print(f"{'  ' * depth}Creating leaf node with value: {leafValue}")
            # Create a leaf node using it
            return Node(current=leafValue)

        # It allows to arrange the features randomly to avoid bias and replace = False takes care of repitition
        featureIndexes = np.random.choice(nFeatures, nFeatures, replace=False)

        # Find the best split
        bestFeature, bestThreshold, bestGain = self.bestSplit(X, y, featureIndexes)

        print(f"{'  ' * depth}Best Split:")
        print(f"{'  ' * depth}Feature: {bestFeature}, Threshold: {bestThreshold:.4f}")
        print(f"{'  ' * depth}Information Gain: {bestGain:.4f}")

        # Create child nodes
        leftIndexes, rightIndexes = self.split(X[:, bestFeature], bestThreshold)
        # The depth + 1 indicated that these new nodes are one level deeper than their parent
        print(f"\n{'  ' * depth}Left split: {len(leftIndexes)} samples")
        left = self.growTree(X[leftIndexes, :], y[leftIndexes], depth + 1)
        print(f"\n{'  ' * depth}Right split: {len(rightIndexes)} samples")
        right = self.growTree(X[rightIndexes, :], y[rightIndexes], depth + 1)

        # Create a node with left and right child
        return Node(bestFeature, bestThreshold, left, right)

    # Method to find the best split
    def bestSplit(self, X, y, featureIndexes):
        # Initialize a variable to store best possible gain
        bestGain = -1
        # Index of the feature used to split and the respective best threshold value
        splitIndex, splitThreshold = None, None

        # Traversing through each feature
        for featureIndex in featureIndexes:
            # Getting all the samples of the selected feature
            XColumn = X[:, featureIndex]
            # Creating an array to store all the unique value of the feature
            thresholds = np.unique(XColumn)

            # Iterating through each value in the above array to find the best threshold value
            for threshold in thresholds:
                # calculating gain using the selected threshold value
                gain = self.informationGain(y, XColumn, threshold)

                # Identifying best gain, its index and the threshold used for it
                if gain > bestGain:
                    bestGain = gain
                    splitIndex = featureIndex
                    splitThreshold = threshold

        # Returning the index used to split and its best threshold
        return splitIndex, splitThreshold, bestGain

    # Method to find information gain
    def informationGain(self, y, XColumn, threshold):
        # Calculating entropy of the parent node
        parentEntropy = self.entropy(y)

        # Split the feature sample based on the threshold
        leftIndex, rightIndex = self.split(XColumn, threshold)

        # Checking whether there are elements in both the variables
        if len(leftIndex) == 0 or len(rightIndex) == 0:
            return 0

        # Length of total sample
        n = len(y)
        # Length of total sample on left and right respectively
        nLeft, nRight = len(leftIndex), len(rightIndex)
        # Entropy of left child and right child
        eLeft, eRight = self.entropy(y[leftIndex]), self.entropy(y[rightIndex])
        # Calculating the child entropy
        childEntropy = (nLeft / n) * eLeft + (nRight / n) * eRight
        # returning information gain which is difference of entropy of parent and child node
        return parentEntropy - childEntropy

    # Method to split the samples based on the selected threshold value
    def split(self, XColumn, splitThreshold):
        # Storing values less than or equal to threshold value on left
        leftindexes = np.argwhere(XColumn <= splitThreshold).flatten()
        # Storing values greater than the threshold value on right
        rightindexes = np.argwhere(XColumn > splitThreshold).flatten()
        # Returning them
        return leftindexes, rightindexes

    # Method to calculate entropy
    def entropy(self, y):
        # Count the occurrences of each integer in the array
        hist = np.bincount(y)
        # Calculate the probability of each class
        ps = hist / len(y)
        # Using the formula of entropy we calculate it (H = -sum(p.log2(p)))
        return -np.sum([p * np.log2(p) for p in ps if p > 0])

    # Method to print the decision tree
    def printTree(self, node, depth=0):
        if node.current is not None:
            print(f"{'  ' * depth}Leaf: {node.current}")
        else:
            print(f"{'  ' * depth}[X{node.feature} <= {node.threshold:.4f}]")
            print(f"{'  ' * depth}Left:")
            self.printTree(node.left, depth + 1)
            print(f"{'  ' * depth}Right:")
            self.printTree(node.right, depth + 1)

    # Method to predict output for test data
    def predict(self, X):
        return np.array([self.traverseTree(x, self.root) for x in X])

    # Method to traverse the input data
    def traverseTree(self, x, node):
        if node.current is not None:
            return node.current

        # Applying condition based on the selected threshold and feature
        if x[node.feature] <= node.threshold:
            return self.traverseTree(x, node.left)
        return self.traverseTree(x, node.right)

In [4]:
# Example usage
if __name__ == "__main__":
    # Generate a simple dataset
    X = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16], [17, 18, 19, 20], [21, 22, 23, 24]])
    y = np.array([0, 1, 0, 1, 1, 0])

    # Create and train the decision tree
    tree = DecisionTree(maxDepth=3)
    tree.fit(X, y)

    # Make predictions
    X_test = np.array([[25, 26, 27, 28], [29, 30, 31, 32]])
    predictions = tree.predict(X_test)
    print("\nPredictions:", predictions)


Depth 0:
Labels: [0 1 0 1 1 0]
Node Entropy: 1.0000
Best Split:
Feature: 2, Threshold: 3.0000
Information Gain: 0.1909

Left split: 1 samples

  Depth 1:
  Labels: [0]
  Node Entropy: -0.0000
  Creating leaf node with value: 0

Right split: 5 samples

  Depth 1:
  Labels: [1 0 1 1 0]
  Node Entropy: 0.9710
  Best Split:
  Feature: 1, Threshold: 18.0000
  Information Gain: 0.3219

  Left split: 4 samples

    Depth 2:
    Labels: [1 0 1 1]
    Node Entropy: 0.8113
    Best Split:
    Feature: 2, Threshold: 11.0000
    Information Gain: 0.3113

    Left split: 2 samples

      Depth 3:
      Labels: [1 0]
      Node Entropy: 1.0000
      Creating leaf node with value: 1

    Right split: 2 samples

      Depth 3:
      Labels: [1 1]
      Node Entropy: -0.0000
      Creating leaf node with value: 1

  Right split: 1 samples

    Depth 2:
    Labels: [0]
    Node Entropy: -0.0000
    Creating leaf node with value: 0

Final Decision Tree Structure:
[X2 <= 3.0000]
Left:
  Leaf: 0
Right:
  